In [18]:
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt
import numpy as np
from time import sleep, time
from pprint import pprint
from prefect import flow, task

In [19]:
EXTERNAL_DATA_PATH = './data/yearly'
AVAILABLE_DATA_PATH = './data/available'

In [22]:
def get_year_from_filename(filepath):
    head, tail = os.path.split(filepath)
    return int(tail.split('.')[0])

@task()
def get_new_dataset(input_folder, latest_year_available = 1900, stream=True):
    '''
    Check if there is new data available and <<download>>.
    If stream is true, only take the oldest <<new files>> available (e.g., if data for 1995 and 1996 is available,
    take only the file for 1995).
    '''
    assert os.path.exists(input_folder), f"Input location {input_folder} does not exist."
    files_list = glob.glob(os.path.join(input_folder, "*.csv"))
    print(f"Found {len(files_list)} data files.")
    valid_files = [file for file in files_list if get_year_from_filename(file) > latest_year_available]
    valid_files.sort()
    print(f"Found {len(valid_files)} data files newer than {latest_year_available}.")
    if len(valid_files) == 0:
        print("No new data available.")
        return None
    if stream:
        valid_files = valid_files[0]
        print(f"Loading only one file: {valid_files}")
    df = pd.concat(map(pd.read_csv, [valid_files]))
    print("Loaded new data.")
    return df

@task()
def get_latest_available_data_year(input_folder):
    assert os.path.exists(input_folder), f"Input location {input_folder} does not exist."
    files_list = glob.glob(os.path.join(input_folder, "*.csv"))
    if len(files_list) == 0:
        return 1900
    else:
        files_list.sort()
        return get_year_from_filename(files_list[-1])

@task()    
def store_data(df, location):
    for year in df['year'].value_counts().index:
        df[df['year'] == year].to_csv(os.path.join(location, f'{year}.csv'), index=False)

@flow(name="main-data-flow")
def main_data_flow(available_data_path, external_data_path):

    # Check the available dataset
    latest_available_data = get_latest_available_data_year(available_data_path)

    # Update dataset        
    df_new = get_new_dataset(external_data_path, latest_year_available=latest_available_data)
        
    if df_new is None:
        print("No new data.")
    else:
        store_data(df_new, location=available_data_path)
        print("Stored new data.")

/usr/local/lib/python3.8/site-packages/prefect/tasks.py:326: UserWarning: A task named 'get_new_dataset' and defined at '/tmp/ipykernel_218/2812644309.py:5' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/usr/local/lib/python3.8/site-packages/prefect/tasks.py:326: UserWarning: A task named 'get_latest_available_data_year' and defined at '/tmp/ipykernel_218/2812644309.py:28' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/usr/local/lib/python3.8/site-packages/prefect/tasks.py:326: UserWarning: A task named 'store_data' and defined at '/tmp/ipykernel_218/2812644309.py:38' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/usr/local/lib/python3.8/site-packages/prefect/flows.py

In [23]:
main_data_flow(available_data_path = AVAILABLE_DATA_PATH, external_data_path = EXTERNAL_DATA_PATH)

17:55:34.670 | INFO    | prefect.engine - Created flow run 'accomplished-caracal' for flow 'main-data-flow'

17:55:35.068 | INFO    | Flow run 'accomplished-caracal' - Created task run 'get_latest_available_data_year-0' for task 'get_latest_available_data_year'

17:55:35.073 | INFO    | Flow run 'accomplished-caracal' - Executing 'get_latest_available_data_year-0' immediately...

17:55:35.359 | INFO    | Task run 'get_latest_available_data_year-0' - Finished in state Completed()

17:55:35.453 | INFO    | Flow run 'accomplished-caracal' - Created task run 'get_new_dataset-0' for task 'get_new_dataset'

...

17:55:36.180 | INFO    | Flow run 'accomplished-caracal' - Executing 'store_data-0' immediately...

17:55:36.573 | INFO    | Task run 'store_data-0' - Finished in state Completed()

Stored new data.


17:55:36.711 | INFO    | Flow run 'accomplished-caracal' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `int`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `DataFrame`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]